In [1]:
# 패키지 로드
import cv2
import mediapipe as mp
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import pickle

# 스쿼트 다운 자세로 classification된 데이터셋 리스트 불러옴
with open("imglist.pkl","rb") as fr:
    squat_down_data = pickle.load(fr)

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
font = cv2.FONT_HERSHEY_SIMPLEX

#Dataset 경로 지정
root_dir = 'C:/Users/user/2022 AI/croped'

#불러올 파일의 확장자 지정(여기서는 이미지 파일만 로드)
img_path_list = []
possible_img_extension = ['.jpg', '.jpeg', '.JPG', '.bmp', '.png']

#Dataset의 경로를 리스트 형태로 가져옴
for (root, dirs, files) in os.walk(root_dir):
    if len(files) > 0:
        for file_name in files:
            if file_name in squat_down_data:
                if os.path.splitext(file_name)[1] in possible_img_extension:
                    img_path = root + '/' + file_name

                    # 경로에서 \를 모두 /로 바꿔줘야 오류가 생기지 않음
                    img_path = img_path.replace('\\', '/') # \는 \\로 나타내야함         
                    img_path_list.append(img_path)
                             


In [2]:

img_path_list

['C:/Users/user/2022 AI/croped/img124_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img132_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img156_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img186_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img209_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img20_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img22_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img275_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img2_119_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img2_129_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img2_133_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img2_161_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img2_170_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img2_269_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img2_273_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img2_274_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img2_285_cropped_.jpg',
 'C:/Users/user/2022 AI/croped/img2_2_cropped_.jpg',
 'C:/Users/user/2022 AI/croped

In [3]:
#불러올 이미지 파일에서 추출할 관절 좌표값을 저장할 리스트 생성
upper_body_angle_list=[]
knee_angle_list=[]
knee_tiptoe_diff_list=[]

In [4]:

# 각도 계산 함수
def calculate_angle(a,b,c):
    a = np.array(a) # 첫 번째 점의 좌표
    b = np.array(b) # 두 번째 점의 좌표
    c = np.array(c) # 세 번째 점의 좌표
    
    # 역탄젠트를 이용해 라디언값을 구해줌
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    # 라디언값을 도단위로 변환
    angle = np.abs(radians*180.0/np.pi)
    
    # 우리는 스쿼트를 하는 신체 각도를 구할 것이므로 180도 이하의 값을 보이도록 보정
    if angle >180.0:
        angle = 360-angle
        
    return angle 


#mediapipe 및 cv를 사용하여 이미지에서 관절 좌표 추출

with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        min_detection_confidence=0.5) as pose:
    
    for idx, file in enumerate(img_path_list):
        image = cv2.imread(file)
        image_height, image_width, _ = image.shape
        h, w = image.shape[:2] 
        # 처리 전 BGR 이미지를 RGB로 변환
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if not results.pose_landmarks:
            continue

        landmarks = results.pose_landmarks.landmark
            
        # mediapipe를 통해 관절 좌표 로드
        left_shoulder = [int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x*w),
                         int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y*h)]
        left_hip = [int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x*w),
                    int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y*h)]
        left_knee = [int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x*w),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y*h)]
        left_ankle = [int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x*w),
                      int(landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y*h)]
        left_heel = [int(landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x*w),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y*h)]
        left_foot_index = [int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x*w),
                           int(landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y*h)]
        right_shoulder = [int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x*w),
                          int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y*h)]
          
        # 정확한 자세인지 확인하기 위해 허리, 무릎의 각도 및 발끝과 무릎 사이의 거리 계산
        upper_body_angle = int((calculate_angle(left_shoulder, left_hip, left_knee)))
        knee_angle = int((calculate_angle(left_hip, left_knee,left_heel)))
        knee_tiptoe_diff = int(left_foot_index[0]-left_knee[0])
        
        # 데이터별 각도를 각 리스트에 append
        upper_body_angle_list.append(upper_body_angle)
        knee_angle_list.append(knee_angle)
        knee_tiptoe_diff_list.append(knee_tiptoe_diff)
        
        # 데이터별 각도를 각 리스트에 append
        upper_body_angle_list.append(upper_body_angle)
        knee_angle_list.append(knee_angle)
        knee_tiptoe_diff_list.append(knee_tiptoe_diff)

#스쿼트 각도 계산에 필요한 허리 각도, 무릎 각도, 무릎과 발끝의 좌표 차이 변수 선언
upper_body_angle_avg=0
knee_angle_avg=0
knee_tiptoe_diff_avg=0

#리스트의 평균을 구하는 함수 생성
def list_average(list):
    avg=0
    sum=0
    for i in list:
        sum+=i
        avg=int(sum/len(list))
    return avg

#허리 각도, 무릎 각도, 무릎과 발끝의 좌표 차이 평균 계산
upper_body_angle_avg=list_average(upper_body_angle_list)
knee_angle_avg=list_average(knee_angle_list)
knee_tiptoe_diff_avg=list_average(knee_tiptoe_diff_list)

In [5]:
upper_body_angle_avg

67

In [6]:
knee_angle_avg

74

In [7]:
knee_tiptoe_diff_avg

-3